# Orquestrador Experimento 1 (apenas dados do IBOVESPA)

In [8]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import datetime
import re
import random

Configurações gerais

In [9]:
NOTEBOOK_FILE = 'Experiment_base.ipynb'
INPUT_WINDOW_SIZE = 40
OUTPUT_WINDOW_SIZE = 5
MAX_EPOCHS = 500
LEARNING_RATE = 1e-2 # 10**-6

FILE = 'data/2 - processed/normalized_data_with_variation.csv'

START_ON_FIRST_VALID_DATE = True
TYPE_OF_UNIT = 'customDays' # "days", "weeks", "months", "percentual", "customDays"
# csv have 7993 data rows
TRAINING_SIZE = '1994-01-03' # 7493 # 7863 # 7593 # .9
VALIDATION_SIZE = '2023-10-01' # 300 # 100 # 200 # .05
TESTING_SIZE = '2024-10-01' # 200 # 30 # 200 # .05

# seed
RANDOM_SEED = random.randint(0, 1000) # 42 # 99
print(f"Random seed: {RANDOM_SEED}")

LOSS_FUNCTION = 'MAE' # 'MSE' # 'MAE'
SAVE_FILE_PATH = 'results/experiment_1'

Random seed: 364


In [10]:
def get_COLUMNS_TO_USE(NUMBER_OF_SUB_EXPERIMENT):
    # define columns to use and columns to predict
    if NUMBER_OF_SUB_EXPERIMENT == 1:
        COLUMNS_TO_USE = ['IBOVESPA - Close']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close'
    if NUMBER_OF_SUB_EXPERIMENT == 2:
        COLUMNS_TO_USE = ['IBOVESPA - Close', 'IBOVESPA - Spread']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close'
    if NUMBER_OF_SUB_EXPERIMENT == 3:
        COLUMNS_TO_USE = ['IBOVESPA - Close', 'IBOVESPA - High', 'IBOVESPA - Low', 'IBOVESPA - Open']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close'
    if NUMBER_OF_SUB_EXPERIMENT == 4:
        COLUMNS_TO_USE = ['IBOVESPA - Close %']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close %'
    if NUMBER_OF_SUB_EXPERIMENT == 5:
        COLUMNS_TO_USE = ['IBOVESPA - Close %', 'IBOVESPA - Spread %']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close %'
    if NUMBER_OF_SUB_EXPERIMENT == 6:
        COLUMNS_TO_USE = ['IBOVESPA - Close %', 'IBOVESPA - High %', 'IBOVESPA - Low %', 'IBOVESPA - Open %']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close %'
    if NUMBER_OF_SUB_EXPERIMENT == 7:
        COLUMNS_TO_USE = ['IBOVESPA - Close', 'IBOVESPA - Close %']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close'
    if NUMBER_OF_SUB_EXPERIMENT == 8:
        COLUMNS_TO_USE = ['IBOVESPA - Close', 'IBOVESPA - Close %', 'IBOVESPA - Spread %']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close'
    if NUMBER_OF_SUB_EXPERIMENT == 9:
        COLUMNS_TO_USE = ['IBOVESPA - Close', 'IBOVESPA - Close %', 'IBOVESPA - High %', 'IBOVESPA - Low %', 'IBOVESPA - Open %']
        COLUMN_TO_PREDICT = 'IBOVESPA - Close'
    return COLUMNS_TO_USE, COLUMN_TO_PREDICT

In [11]:
def update_RUN_MOMENT(nb, RUN_MOMENT):
    """
    Update the RUN_MOMENT variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("RUN_MOMENT =.*?(?=\n)", f"RUN_MOMENT = \"{RUN_MOMENT}\"", nb.cells[4]['source'])

def update_SUB_EXPERIMENT(nb, NUMBER_OF_SUB_EXPERIMENT):
    """
    Update the NUMBER_OF_SUB_EXPERIMENTS variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("NUMBER_OF_SUB_EXPERIMENT =.*?(?=\n)", f"NUMBER_OF_SUB_EXPERIMENT = {NUMBER_OF_SUB_EXPERIMENT}", nb.cells[4]['source'])
    COLUMNS_TO_USE, COLUMN_TO_PREDICT = get_COLUMNS_TO_USE(NUMBER_OF_SUB_EXPERIMENT)
    nb.cells[4]['source'] = re.sub("COLUMNS_TO_USE =.*?(?=\n)", f"COLUMNS_TO_USE = {COLUMNS_TO_USE}", nb.cells[4]['source'])
    nb.cells[4]['source'] = re.sub("COLUMN_TO_PREDICT =.*?(?=\n)", f"COLUMN_TO_PREDICT = '{COLUMN_TO_PREDICT}'", nb.cells[4]['source'])

def update_MODEL_TO_USE(nb, MODEL_TO_USE):
    """
    Update the MODEL_TO_USE variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("MODEL_TO_USE =.*?(?=\n)", f"MODEL_TO_USE = {MODEL_TO_USE}", nb.cells[4]['source'])

def update_WINDOW_SIZE(nb, INPUT_WINDOW_SIZE, OUTPUT_WINDOW_SIZE):
    """
    Update the INPUT_WINDOW_SIZE and OUTPUT_WINDOW_SIZE variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("INPUT_WINDOW_SIZE =.*?(?=\n)", f"INPUT_WINDOW_SIZE = {INPUT_WINDOW_SIZE}", nb.cells[4]['source'])
    nb.cells[4]['source'] = re.sub("OUTPUT_WINDOW_SIZE =.*?(?=\n)", f"OUTPUT_WINDOW_SIZE = {OUTPUT_WINDOW_SIZE}", nb.cells[4]['source'])

def update_MAX_EPOCHS(nb, MAX_EPOCHS):
    """
    Update the MAX_EPOCHS variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("MAX_EPOCHS =.*?(?=\n)", f"MAX_EPOCHS = {MAX_EPOCHS}", nb.cells[4]['source'])
    
def update_LEARNING_RATE(nb, LEARNING_RATE):
    """
    Update the LEARNING_RATE variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("LEARNING_RATE =.*?(?=\n)", f"LEARNING_RATE = {LEARNING_RATE}", nb.cells[4]['source'])

def update_DF_SIZEs(nb, TYPE_OF_UNIT, TRAINING_SIZE, VALIDATION_SIZE, TESTING_SIZE):
    """
    Update the TYPE_OF_UNIT, TRAINING_SIZE, VALIDATION_SIZE and TESTING_SIZE variables in the notebook.
    """
    nb.cells[4]['source'] = re.sub("TYPE_OF_UNIT =.*?(?=\n)", f"TYPE_OF_UNIT = '{TYPE_OF_UNIT}'", nb.cells[4]['source'])
    if TYPE_OF_UNIT == 'customDays':
        nb.cells[4]['source'] = re.sub("TRAINING_SIZE =.*?(?=\n)", f"TRAINING_SIZE = '{TRAINING_SIZE}'", nb.cells[4]['source'])
        nb.cells[4]['source'] = re.sub("VALIDATION_SIZE =.*?(?=\n)", f"VALIDATION_SIZE = '{VALIDATION_SIZE}'", nb.cells[4]['source'])
        nb.cells[4]['source'] = re.sub("TESTING_SIZE =.*?(?=\n)", f"TESTING_SIZE = '{TESTING_SIZE}'", nb.cells[4]['source'])
    else:
        nb.cells[4]['source'] = re.sub("TRAINING_SIZE =.*?(?=\n)", f"TRAINING_SIZE = {TRAINING_SIZE}", nb.cells[4]['source'])
        nb.cells[4]['source'] = re.sub("VALIDATION_SIZE =.*?(?=\n)", f"VALIDATION_SIZE = {VALIDATION_SIZE}", nb.cells[4]['source'])
        nb.cells[4]['source'] = re.sub("TESTING_SIZE =.*?(?=\n)", f"TESTING_SIZE = {TESTING_SIZE}", nb.cells[4]['source'])

def update_RANDOM_SEED(nb, RANDOM_SEED):
    """
    Update the RANDOM_SEED variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("RANDOM_SEED =.*?(?=\n)", f"RANDOM_SEED = {RANDOM_SEED}", nb.cells[4]['source'])

def update_LOSS_FUNCTION(nb, LOSS_FUNCTION):
    """
    Update the LOSS_FUNCTION variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("LOSS_FUNCTION =.*?(?=\n)", f"LOSS_FUNCTION = '{LOSS_FUNCTION}'", nb.cells[4]['source'])

def update_SAVE_FILE_PATH(nb, SAVE_FILE_PATH):
    """
    Update the SAVE_FILE_PATH variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("SAVE_FILE_PATH =.*?(?=\n)", f"SAVE_FILE_PATH = '{SAVE_FILE_PATH}'", nb.cells[4]['source'])

def update_START_ON_FIRST_VALID_DATE(nb, START_ON_FIRST_VALID_DATE):
    """
    Update the START_ON_FIRST_VALID_DATE variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("START_ON_FIRST_VALID_DATE =.*?(?=\n)", f"START_ON_FIRST_VALID_DATE = {START_ON_FIRST_VALID_DATE}", nb.cells[4]['source'])

def update_FILE(nb, FILE):
    """
    Update the FILE variable in the notebook.
    """
    nb.cells[4]['source'] = re.sub("FILE =.*?(?=\n)", f"FILE = '{FILE}'", nb.cells[4]['source'])

# Run Experiments

In [12]:
for i in range(1, 4): # 1 to 2
    RANDOM_SEED = random.randint(0, 1000) # 42 # 99
    print(f"Random seed: {RANDOM_SEED}")
    for sub in range(1, 10): # 1 to 9
        NUMBER_OF_SUB_EXPERIMENT = sub
        for model in range(1, 4): # 1 to 3
            MODEL_TO_USE = model
            RUN_MOMENT = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            print(f"RUN_MOMENT: {RUN_MOMENT} - Sub: {NUMBER_OF_SUB_EXPERIMENT} - Model: {MODEL_TO_USE}")

            # Load the notebook
            with open(NOTEBOOK_FILE) as f:
                nb = nbformat.read(f, as_version=4)

            # Config the notebook to use the correct kernel
            ep = ExecutePreprocessor(kernel_name='ibovespa-env')

            # Update the notebook with the new parameters
            update_RUN_MOMENT(nb, RUN_MOMENT)
            update_SUB_EXPERIMENT(nb, NUMBER_OF_SUB_EXPERIMENT)
            update_MODEL_TO_USE(nb, MODEL_TO_USE)
            update_WINDOW_SIZE(nb, INPUT_WINDOW_SIZE, OUTPUT_WINDOW_SIZE)
            update_MAX_EPOCHS(nb, MAX_EPOCHS)
            update_LEARNING_RATE(nb, LEARNING_RATE)
            update_DF_SIZEs(nb, TYPE_OF_UNIT, TRAINING_SIZE, VALIDATION_SIZE, TESTING_SIZE)
            update_RANDOM_SEED(nb, RANDOM_SEED)
            update_LOSS_FUNCTION(nb, LOSS_FUNCTION)
            update_SAVE_FILE_PATH(nb, SAVE_FILE_PATH)
            update_START_ON_FIRST_VALID_DATE(nb, START_ON_FIRST_VALID_DATE)

            # Execute the notebook
            ep.preprocess(nb)

            # Save the updated notebook
            with open(f"{SAVE_FILE_PATH}/notebooks/{RUN_MOMENT}-s{NUMBER_OF_SUB_EXPERIMENT}-m{MODEL_TO_USE}.ipynb", 'w', encoding='utf-8') as f:
                nbformat.write(nb, f)

Random seed: 280
RUN_MOMENT: 2025-04-14 21:00:15 - Sub: 1 - Model: 1


2025-04-14 21:00:18.454174: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 21:02:57.219816: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-14 21:02:57.468797: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-14 21:03:03.608125: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


RUN_MOMENT: 2025-04-14 21:03:11 - Sub: 1 - Model: 2


2025-04-14 21:03:13.982077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 21:05:12.874563: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-14 21:05:13.197960: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-14 21:05:21.087883: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


RUN_MOMENT: 2025-04-14 21:05:29 - Sub: 1 - Model: 3


...

2025-04-15 01:07:30.054621: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 01:09:22.883575: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-15 01:09:23.140166: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-15 01:09:29.492898: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


RUN_MOMENT: 2025-04-15 01:09:36 - Sub: 9 - Model: 2


2025-04-15 01:09:39.000606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 01:11:40.596111: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-15 01:11:40.828457: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-15 01:11:48.391438: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


RUN_MOMENT: 2025-04-15 01:11:57 - Sub: 9 - Model: 3


2025-04-15 01:12:00.227062: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 01:15:40.596876: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-15 01:15:40.824434: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-15 01:15:49.528900: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
